# Proyecto 1: Analítica de textos

## Miembros
- Estudainte 3: Mariana Pineda Miranda - 202123330
- Estudiante 2: Isaac David Bermúdez- 202014146
- Estudiante 1: Joseph Linares Gutierrez - 202111887

In [134]:
import re
import numpy as np
import pandas as pd
from collections import Counter


from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats

pd.set_option('display.max_columns', None)   # todas las columnas
pd.set_option('display.width', None) 

## Perfilamiento y análisis de datos

In [135]:
dataTrain = pd.read_csv("data/train.csv")
dataTest = pd.read_csv("data/test.csv")

In [136]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   textos  2424 non-null   object
 1   labels  2424 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 38.0+ KB


In [137]:
dataTrain['textos'].value_counts()

"Aprendizaje" y "educación" se consideran sinónimos de escolarización formal. Las organizaciones auxiliares, como las editoriales de educación, las juntas examinadoras y las organizaciones de formación de docentes, se consideran extensiones de los acuerdos establecidos por los gobiernos. Este marco de comprensión se ha vuelto cada vez más inadecuado.                                                                                                                                                                                                                                                                                                                                                                                                                       1
El arrendamiento de grandes porciones de estas tierras consuetudinarias a propietarios privados afecta a la seguridad alimentaria y a todas las actividades de subsistencia de quienes antes dependían de estos recursos naturales. Los despose

In [138]:
dataTrain['labels'].value_counts()

4    1025
3     894
1     505
Name: labels, dtype: int64

In [139]:
dataTrain['textos'].str.len().describe()

count    2424.000000
mean      683.223185
std       223.563695
min       303.000000
25%       513.000000
50%       647.000000
75%       809.250000
max      1492.000000
Name: textos, dtype: float64

In [140]:
dataTrain.isnull().sum()

textos    0
labels    0
dtype: int64

## Tratamiento de Datos

#### Descarga de las stopwords en español

In [141]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [142]:
from nltk.corpus import stopwords

# Lista de stopwords en español
stopwordsInSpanish = stopwords.words('spanish')

print(len(stopwordsInSpanish))
print(stopwordsInSpanish)

313
['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están'

#### Limpieza y vectorización de datos

In [ ]:
def limpiar_texto(texto: str) -> str:
    texto = texto.lower()                               # Convertir a minusculas
    texto = re.sub(r'[^a-záéíóúüñ\s]', '', texto)       # Eliminar caracteres especiales y numeros
    return texto

dataTrain['texto_limpio'] = dataTrain['textos'].apply(limpiar_texto)

In [144]:
texto_limpio = ' '.join(dataTrain['texto_limpio']).split()

counter = Counter(texto_limpio) # Contar frecuencia
print(counter.most_common(50))

[('de', 22276), ('la', 12254), ('en', 9210), ('los', 8551), ('y', 8101), ('el', 6874), ('que', 5043), ('a', 4768), ('las', 4651), ('para', 3491), ('se', 2897), ('del', 2754), ('un', 2446), ('por', 2237), ('con', 2151), ('una', 2135), ('más', 1797), ('es', 1358), ('como', 1294), ('no', 1226), ('al', 1153), ('salud', 1108), ('pobreza', 1020), ('educación', 995), ('países', 963), ('entre', 962), ('atención', 900), ('o', 798), ('también', 767), ('estudiantes', 742), ('sobre', 674), ('su', 671), ('lo', 629), ('son', 604), ('ocde', 587), ('escuelas', 586), ('años', 560), ('servicios', 549), ('este', 533), ('ha', 523), ('nivel', 521), ('sistema', 508), ('sus', 500), ('sin', 499), ('niños', 481), ('personas', 457), ('pueden', 453), ('calidad', 447), ('han', 437), ('desarrollo', 431)]


#### Adicionar y eliminar stopwords

In [ ]:
# Listas de palabras a conservar (eliminar de stopwords)
palabras_de_Negacion = ['no', 'ni', 'nunca', 'jamás', 'tampoco',
                        'sin', 'ningún', 'ninguna', 'ninguno', 
                        'ningunos', 'ningunas', 'nada', 'nadie',
                        'hay    ', 'había', 'hubo', 'falta', 'faltan',
                        'faltar', 'carecer', 'carecen', 'carencia', 
                        'carencias', 'escasez', 'ausencia', 'inaccesible', 'inaccesibilidad']

palabras_de_poder = ['puedo', 'puede', 'pueden', 'podemos', 'podría', 'podrían']

# Listas de palabras a agregar (adicionar a stopwords)
palabras_conectoras = ['y', 'e', 'además', 'también', 'incluso', 
                       'más', 'pero', 'aunque', 'sino']


stopwordsInSpanish = [w for w in stopwordsInSpanish if w not in palabras_de_Negacion and w not in palabras_de_poder]
stopwordsInSpanish.extend(palabras_conectoras)

print(len(stopwordsInSpanish))
print(stopwordsInSpanish)

325
['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sobre', 'también', 'me', 'hasta', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'esté', 'estés', 'estemos', 'est

In [ ]:
def limpiar_stopword(w):
    return re.sub(r'[^a-záéíóúüñ\s]', '', w.lower())

stopwordsInSpanish = [limpiar_texto(w) for w in stopwordsInSpanish]

vectorizer = TfidfVectorizer(stop_words = stopwordsInSpanish, max_features=1000)
X = vectorizer.fit_transform(dataTrain['texto_limpio'])
y = dataTrain['labels']